<a href="https://colab.research.google.com/github/jko6422/testfile/blob/main/test_2021_08_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특수문자 손글씨 인식 프로그램

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/

# **데이터 세트 제작 파트**

## 함수 모음집. 최초 코드 기동 시 전부 실행하고 진행할 것!

#####기본 함수

In [ ]:
# 1. 최초 설정 함수

def setting() : 
    #그레이 스케일로 변환 
    src = cv2.imread(filename,cv2.IMREAD_GRAYSCALE) 

    #이미지의 크기를 비율 맞춰 조정.
    #기본 수치 744. 안되면 740으로 해보고 그래도 안되면 임의조정.

    changed_width = 744
    n = (src.shape[1] / changed_width)

    changed_height = (src.shape[0] / n)
    src = cv2.resize(src , ( changed_width,int(changed_height) ) ) 

    #양 옆의 노이즈를 제거. 불필요할 시 각주처리할 것.
    #src = src[0:src.shape[0]-10, 5:src.shape[1]-10]

    #영상 이진화 
    ret , binary = cv2.threshold(src,170,255,cv2.THRESH_BINARY_INV) 

    #외곽선 검출
    contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
 
    #이진화 이미지를 color로 복사한다.
    color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

    #초록색으로 외곽선을 그려준다.
    cv2.drawContours(color , contours , -1 , (0,255,0),3)

    """
    검출한 외곽선에 사각형을 그려서 배열에 추가. 이때 크기가 작은 노이즈는 1차로 거른다.
    boundingRect : 외각선을 감싸는 최소 크기 사각형 반환. x, y는 좌표. w는 폭, h는 넓이.
    append : 리스트에 요소 추가
    """
    bR_arr = [] 

    for i in range(len(contours)) : 
        bin_tmp = binary.copy() 
        x,y,w,h = cv2.boundingRect(contours[i])
        if (x and y >= 2) and (h > 1) and (w and h < 100): 
            bR_arr.append([x,y,w,h])

    return bR_arr, bin_tmp, color, src

# 2. 네모 그리기 함수

"""
# 데이터에 사각형그리기. #y축을 기준으로 10개씩 리스트로 묶어서 저장 
# 순서는 !부터 위에서 아래로 10번(count). 그 뒤에 다음 이모티콘을 저장하는 방식.(county)
# countbox는 네모가 쳐진 갯수. 100개면 성공.
"""

def drawrec(bR_arr, color) : 
    # 변수 초기화
    digit_arr = [] 
    digit_arr2 = []
    for n in range(10) : 
        digit_arr2.append([])

    count = 0
    countbox = 0

    # y축으로 정렬
    bR_arr = sorted(bR_arr, key=lambda num : num[1], reverse = False)

    n = 0
    for n in range(10) : 
        bR_arr2 = []
        n1 = (n * 10)
        n2 = n1 + 10

        bR_arr2 = (bR_arr[n1:n2]) 
        bR_arr2 = sorted(bR_arr2, key=lambda num : (num[0]+num[2]/2), reverse = False)

        # 검증. 배열이 정상적으로 들어가는 중인가?
        if  abs(int(bR_arr2[0][0]) - int(bR_arr2[1][0])) < 30 : 
            print("배열 이상 확인, x축 정렬로 이행.")
            return 0

        print(bR_arr2[0:10])

        #자르기!
        for x,y,w,h in bR_arr2 : 
            if y ==0 : 
                tmp_y = bin_tmp[0:y+h+4,x-2:x+w+2].shape[0] #y축 계산
            else :
                tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
            
            tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산

            # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
            # %,는 크기를 넓게 세팅. !와 ?는 x축만 조금 보정.
            # 그 이외는 글자 크기에 맞게 세팅.
            if (tmp_x and tmp_y > 10) or count == 5 or count == 7 : 
                if count == 0 or count == 8: 
                    if x-4 < 0 :
                        cv2.rectangle(color,(0,y-2),(x+w+4,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-2:y+h+2,0:x+w+4]) 
                    else : 
                        cv2.rectangle(color,(x-4,y-2),(x+w+4,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-2:y+h+2,x-4:x+w+4]) 
                elif count == 4 : 
                    if y-10 < 0 : 
                        cv2.rectangle(color,(x-10,0),(x+w+10,y+h+10),(0,0,255),1) 
                        digit_arr.append(bin_tmp[0:y+h+10,x-10:x+w+10])
                    else : 
                        cv2.rectangle(color,(x-10,y-10),(x+w+10,y+h+10),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-10:y+h+10,x-10:x+w+10])
                else : 
                    if y-2 < 0 : 
                        cv2.rectangle(color,(x-2,0),(x+w+2,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[0:y+h+2,x-2:x+w+2]) 
                    else : 
                        cv2.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2]) 

                digit_arr2[count].append(digit_arr[0])
                digit_arr = []
                count += 1
                countbox += 1
 
                if count == 10 :
                    count = 0
 
    print("실제로 그어진 네모의 개수 : ", countbox)
    return digit_arr2

# x축 기준 저장.
def drawrec2(bR_arr, color) :

    digit_arr = [] 
    digit_arr2 = [] 

    count = 0
    county = 0
    countbox = 0

    #확실한 배열 순서를 위해 x + w/2를 기준으로 하여 정렬하도록 설정
    bR_arr = sorted(bR_arr, key=lambda num : (num[0] + num[2]/2), reverse = False)

    # 배열 확인
    for n in range(10) : 
        n1 = (n * 10)
        n2 = n1 + 10
        print(bR_arr[n1:n2])

    # 자르기!
    for x,y,w,h in bR_arr : 
        tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
        tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산

        # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
        # %,는 크기를 넓게 세팅. !와 ?는 x축만 조금 보정.
        # 그 이외는 글자 크기에 맞게 세팅.
        if (tmp_x and tmp_y > 10) or county == 5 or county == 7 : 
                if county == 0 or county == 8: 
                    cv2.rectangle(color,(x-4,y-2),(x+w+4,y+h+2),(0,0,255),1) 
                    digit_arr.append(bin_tmp[y-2:y+h+2,x-4:x+w+4]) 
                elif county == 4 : 
                    if y-10 < 0 : 
                        cv2.rectangle(color,(x-10,0),(x+w+10,y+h+10),(0,0,255),1) 
                        digit_arr.append(bin_tmp[0:y+h+10,x-10:x+w+10])
                    else : 
                        cv2.rectangle(color,(x-10,y-10),(x+w+10,y+h+10),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-10:y+h+10,x-10:x+w+10])
                else : 
                    if y-2 < 0 : 
                        cv2.rectangle(color,(x-2,0),(x+w+2,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[0:y+h+2,x-2:x+w+2]) 
                    else : 
                        cv2.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1) 
                        digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2]) 

                count += 1
                countbox += 1

                if count == 10 : 
                    digit_arr2.append(digit_arr) 
                    digit_arr = [] 
                    count = 0
                    county += 1
                
                if county == 10 :
                    break

    print("실제로 그어진 네모의 개수 : ", countbox)
    return digit_arr2, color

#####인식 보정 함수 모음

In [ ]:
# 인식용 범위 설정
def changesetting(bR_arr) : 
    erase_dot = []
    count = 0
    county = 0

    # x를 기준으로 정렬
    bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)

    # 네모가 그려질 위치를 미리 파악한다.
    for x,y,w,h in bR_arr : 
        tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0] #y축 계산
        tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1] #x축 계산
        # %는 크기를 넓게 세팅.
        # ^ 와 * 은 크기가 작으므로 조건없이 별도로 처리하게 세팅.
        # 그 이외는 글자 크기에 맞게 세팅.
        if (tmp_x and tmp_y > 20) or county == 5 or county == 7 : 
            erase_dot.append([x,y]) 
            count += 1

            if count == 10 : 
                count = 0
                county += 1

    return erase_dot

# 1. 느낌표 인식 보정
def firstchange(bR_arr,erase_dot) : 
    erase_dot_x_1 = 0

    # 1은 느낌표 범위
    erase_dot = sorted(erase_dot, key=lambda num : num[0], reverse = False)
    erase_dot_x_1 = erase_dot[9][0] + 10
    print("느낌표 적용범위", erase_dot_x_1)

    count = 0

    for i in range(0, len(bR_arr)-1) :
        # y축 차이 계산. 배열수 감소로 에러 시 break하고 다음 i로 이동하게 설정.
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                x_difference = int(bR_arr[i][0]) - int(bR_arr[j][0])
                y_difference = int(bR_arr[i][1]) + int(bR_arr[i][3]) - int(bR_arr[j][1]) - int(bR_arr[j][3])
            except : 
                break

            if int(bR_arr[i][0] ) <= erase_dot_x_1 and int(bR_arr[j][0]) <= erase_dot_x_1 : 
                # x, y의 길이 차가 별로 안 날 것.
                if abs(x_difference) <= 20 and abs(y_difference) <= 50 : 

                    # 높이 비교로 막대기 부분 점 부분 판정.
                    if int(bR_arr[i][3]) < int(bR_arr[j][3]) :
                        # [i]가 점, [j]가 막대기
                        if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                            bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1] 

                            if bR_arr[i][0] < bR_arr[j][0] : 
                                bR_arr[j][0] = bR_arr[i][0]
                                bR_arr[j][2] += bR_arr[i][2]

                            del bR_arr[i]
                            count += 1

                    elif int(bR_arr[i][3]) > int(bR_arr[j][3]) : 
                        # [i]가 막대기, [j]가 점
                        if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                            bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                            if bR_arr[j][0] < bR_arr[i][0] : 
                                bR_arr[i][0] = bR_arr[j][0]
                                bR_arr[i][2] += bR_arr[j][2]

                            del bR_arr[j]
                            count += 1

    print("느낌표 보정 적용 수 : ", count)


# 2번. 물음표 인식 보정
def secondchange(bR_arr,erase_dot) : 
    erase_dot_x_2 = 0
    erase_dot_x_3 = 0

    # 물음표 보정 수치.
    length = len(erase_dot) - 100

    # 2,3은 물음표 범위
    for i in range (69+length,len(erase_dot)-1) : 
        if erase_dot[i+1][0] - erase_dot[i][0] >10 :
            erase_dot_x_2 = erase_dot[i+1][0] - 20
            break

    for i in range (89+length,len(erase_dot)-1) : 
        if erase_dot[i+1][0] - erase_dot[i][0] >10 :
            erase_dot_x_3 = erase_dot[i+1][0] + 5
            break

    if erase_dot_x_3 == 0 :
        print("범위 이상 발생으로 임의 범위 설정")
        erase_dot_x_3 = erase_dot[len(erase_dot)-1][0]

    print("물음표 적용범위", erase_dot_x_2, erase_dot_x_3)

    count = 0
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) :
            try : 
                x_difference = int(bR_arr[i][0]) - int(bR_arr[j][0])
                y_difference = int(bR_arr[i][1]) + int(bR_arr[i][3]) - int(bR_arr[j][1]) - int(bR_arr[j][3])
            except : 
                break

            if int(bR_arr[i][0] ) >= erase_dot_x_2  and int(bR_arr[j][0]) >= erase_dot_x_2  :
                if int(bR_arr[i][0] ) <= erase_dot_x_3  and int(bR_arr[j][0]) <= erase_dot_x_3  : 

                    # x와 y의 길이 차가 별로 안 날 것.
                    if abs(x_difference) <= 25 and abs(y_difference) <= 30: 
                        # 높이 비교로 막대기 부분 점 부분 판정.

                        if int(bR_arr[i][3]) < int(bR_arr[j][3]) :
                            # [i]가 점, [j]가 막대기
                        
                            if int(bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]) > 0 :
                                bR_arr[j][3] = bR_arr[i][3] + bR_arr[i][1] - bR_arr[j][1]

                                if bR_arr[i][0] < bR_arr[j][0] : 
                                    bR_arr[j][0] = bR_arr[i][0]
                                    bR_arr[j][2] += bR_arr[i][2]

                                del bR_arr[i]
                                count += 1

                        elif int(bR_arr[i][3]) > int(bR_arr[j][3]) :                        
                            # [i]가 막대기, [j]가 점

                            if int(bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]) > 0:
                                bR_arr[i][3] = bR_arr[j][3] + bR_arr[j][1] - bR_arr[i][1]

                                if bR_arr[j][0] < bR_arr[i][0] : 
                                    bR_arr[i][0] = bR_arr[j][0]
                                    bR_arr[i][2] += bR_arr[j][2]

                                del bR_arr[j]
                                count += 1

    print("물음표 보정 적용 수 : ", count)
    
    """   
    print("물음표 배열")
    for i in range(0, len(bR_arr)-1) : 
        if int(bR_arr[i][0] ) >= erase_dot_x_2 -1 and int(bR_arr[i][0] ) <= erase_dot_x_3 +1:
            print(bR_arr[i])
    """


# 3번 % 문자 보정 및 근처 노이즈 제거

"""
 글자 @와 % 를 보정하자.
 직사각형을 그릴 시, 특수문자 전체만이 아닌 일부까지 인식하는 것을 막도록 노이즈 배열을 제거한다.
 큰 네모와 작은 네모가 있을 때 x 좌표와 y 좌표를 비교 후(x_big < x_small 이고 y_big < y_small 일 것. x와 y를 비교)
 작은 네모가 큰 네모 안에 있는지 비교하고 (x_big_1 > x_small_1 이고 y_big_1 > y_small_1일 것. x+w와 y+h를 비교)
 조건이 맞을 시 작은 네모로 판정된 배열을 제거한다.
"""
def thirdchange(bR_arr) : 
    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) : 

        #베열 파악 중 오류 발생 시 j for문을 break 후 다음 i로 넘기도록 세팅.
        # i가 큰 네모, j가 작은 네모라 가정 후 판단
            try : 
                x_big = int(bR_arr[i][0]) - 10
                y_big = int(bR_arr[i][1]) - 10
                x_small = int(bR_arr[j][0]) 
                y_small = int(bR_arr[j][1])
            except : 
                continue

            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[i][0] + bR_arr[i][2]) + 10
                y_big_1 = int(bR_arr[i][1] + bR_arr[i][3]) + 10
                x_small_1 = int(bR_arr[j][0] + bR_arr[j][2])
                y_small_1 = int(bR_arr[j][1] + bR_arr[j][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[j]

        # j가 큰 네모, i가 작은 네모라 가정 후 판단
            try : 
                x_big = int(bR_arr[j][0]) - 10
                y_big = int(bR_arr[j][1]) - 10
                x_small = int(bR_arr[i][0]) 
                y_small = int(bR_arr[i][1])
            except : 
                break

            if x_big < x_small and y_big < y_small : 
                x_big_1 = int(bR_arr[j][0] + bR_arr[j][2]) + 10
                y_big_1 = int(bR_arr[j][1] + bR_arr[j][3]) + 10
                x_small_1 = int(bR_arr[i][0] + bR_arr[i][2])
                y_small_1 = int(bR_arr[i][1] + bR_arr[i][3])

                if x_big_1 > x_small_1 and y_big_1 > y_small_1 :
                    del bR_arr[i]

# 4번 %문자 노이즈 제거 2
def thirdchange1(bR_arr,erase_dot) : 
    length = len(erase_dot) - 100

    # 4,5는 %의 범위
    for i in range (35,len(erase_dot)-1) : 
        if erase_dot[i+1][0] - erase_dot[i][0] > 10 :
            erase_dot_x_4 = erase_dot[i+1][0] - 20
            break

    for i in range (49+length,len(erase_dot)-1) : 
        if erase_dot[i+1][0] - erase_dot[i][0] >10 :
            erase_dot_x_5 = erase_dot[i+1][0] + 5
            break

    print("% 적용범위", erase_dot_x_4, erase_dot_x_5)

    for i in range(0, len(bR_arr)-1) :
        for j in range(i+1, len(bR_arr)-1) : 
            try : 
                x_difference = int(bR_arr[i][0]) - int(bR_arr[j][0])
                y_difference = int(bR_arr[i][1]) + int(bR_arr[i][3]) - int(bR_arr[j][1]) - int(bR_arr[j][3])
            except : 
                break
            
            if int(bR_arr[i][0] ) >= erase_dot_x_4  and int(bR_arr[j][0]) >= erase_dot_x_4  :
                if int(bR_arr[i][0] ) <= erase_dot_x_5  and int(bR_arr[j][0]) <= erase_dot_x_5  : 
                    if abs(x_difference) <= 30 and abs(y_difference) <= 30:
                        if bR_arr[i][3] > bR_arr[j][3] : 
                            del bR_arr[j]
                        elif bR_arr[j][3] > bR_arr[i][3] : 
                            del bR_arr[i]

# 5번 최종 노이즈 제거
def deletenoise(bR_arr) : 
    for i in range(len(bR_arr)) : 
        try : 
            if bR_arr[i][3] < 2 : 
                del bR_arr[i]
        except : 
            break

    print("최종적으로 남은 배열 수 : ", len(bR_arr))

##데이터 세트 글자 구분

In [ ]:
#필요 함수 import
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

"""
원활한 이미지 인식을 위한 준비 작업.
"""

# 이미지 파일 경로 지정. 이미지는 camscanner로 흑색보정 후 드라이브에 업로드한다. JPG 일 것!
#103
filename = 'A4/95.jpg'

# 최초 설정
bR_arr, bin_tmp, color, src = setting()

"""
문자 인식 보정
"""

# 인식용 범위 설정
dot = changesetting(bR_arr)


# 느낌표. 3번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
firstchange(bR_arr,dot)
firstchange(bR_arr,dot)
firstchange(bR_arr,dot)

# 물음표. 2번 반복!
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
secondchange(bR_arr,dot)
secondchange(bR_arr,dot)

# %와 기타 노이즈 보정. y로 정렬하고 실행 뒤, x로 정렬해서 한번 더!
bR_arr = sorted(bR_arr, key=lambda num : num[1], reverse = False)
thirdchange(bR_arr)
bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
thirdchange(bR_arr)

"""
!!!!!인식이 잘 안될때!!!!!
@, %가 인식이 잘 안되면 아래 주석을 해제해주세요!
"""
thirdchange1(bR_arr,dot)

# 최종 노이즈 제거
deletenoise(bR_arr)

"""
자르기!
"""

# 자르기 실시. 이상 감지 시 x축 정렬로 재시도.
digit_arr2 = drawrec(bR_arr, color)
if digit_arr2 == 0 : 
    # 그림판 갈아엎기
    bR_arr_notuse, bin_tmp_notuse, color, src = setting()
    digit_arr2, color = drawrec2(bR_arr, color)

#결과 확인
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

데이터 세트 저장

In [ ]:
import os

#리스트에 저장된 이미지를 32x32의 크기로 리사이즈해서 순서대로 저장 
for i in range(0,len(digit_arr2)) : 
    for j in range(len(digit_arr2[i])) :
     
        # !과 ?의 경우 비율 유지를 위한 마스크 생성
        print(i,j)
        print(digit_arr2[i][j].shape[1],digit_arr2[i][j].shape[0] )

        if i == 0 or i==8: 
            width = digit_arr2[i][j].shape[1]
            height = digit_arr2[i][j].shape[0] 
            tmp = (height - width)/2
            mask = np.zeros((height,height)) 
            mask[0:height,int(tmp):int(tmp)+width] = digit_arr2[i][j]
            digit_arr2[i][j] = cv2.resize(mask,(32,32)) 
        else:
            digit_arr2[i][j] = cv2.resize(digit_arr2[i][j],(32,32))

# 저장 경로 지정
path = 'data/'

# 파일 갯수에 따라 자동으로 순번을 매기도록 한다!
# 첫번째 숫자는 라벨, 두번째 숫자는 그림의 순번을 나타낸다.
file_list = os.listdir(path)
nextfile = len(file_list) / 10
#nextfile = 930

print("현재 파일 갯수 : ", len(file_list))
print("시작 순번", int(nextfile))

for i in range(0,len(digit_arr2)) : 
  for j in range(len(digit_arr2[i])) :
      j1 = int(j + nextfile)
      cv2.imwrite(path + str(i)+'_'+str(j1)+'.png',digit_arr2[i][j])

file_list = os.listdir(path)
print("최종 파일 갯수 : ", len(file_list))

# **데이터 모델 제작 파트**

##저장된 데이터 세트 분류

데이터 세트 제작

In [ ]:
#회전시키자
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("data/3_0.png", cv2.IMREAD_COLOR)
 
cv2_imshow(img)
print("")

# -20~20
n=0
height, width, channel = img.shape
for i in range(4) : 
    if i ==0 : 
        n = -20
    elif i ==1 : 
        n = -10
    elif i ==2 : 
        n = 10
    elif i ==3 : 
        n = 20
    
    matrix = cv2.getRotationMatrix2D((width/2, height/2),n, 1)
    dst = cv2.warpAffine(img, matrix, (width, height))
    cv2_imshow(dst)
    print("")

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 파일이름체크로 데이터 세트 분류!
import os 
import cv2
import numpy as np
from sklearn.preprocessing import * 
import matplotlib.pyplot as plt 

# 데이터 세트에서 훈련용 데이터가 가지는 비율 (90%면 0.9입니다)
rate = 0.9

def createdataset(directory):       #sklearn사용을 위해 데이터세트를 생성

    # 파일 불러오기
    files = os.listdir(directory)

    # 훈련용 배열
    x = []
    y = []

    # 테스트용 베열
    x1 = []
    y1 = []

    # 테스트용 파일의 시작 순번
    file_num = int(len(files) / 10 * rate)
    
    print("저장된 파일 수 : ", len(files))
    print("훈련용 파일의 마지막 순번 : ", file_num)
    print("예상되는 훈련용 파일 수 : ", file_num * 10)

    for file in files:
        attr_x = cv2.imread(directory+file, cv2.IMREAD_GRAYSCALE)
        attr_x = attr_x.reshape(32,32,1)
        attr_y = int(file[0])
        
        #파일 뒤의 숫자를 확인하여 훈련용, 테스트용을 비율로 구분한다.
        numbercheck = int(file.split('_')[1].split('.')[0])
       
        # 훈련용으로 분류
        if numbercheck < file_num : 
            x.append(attr_x)
            y.append(attr_y)
            print(numbercheck, attr_y, "훈련용")

        # 테스트용으로 분류
        else : 
            x1.append(attr_x)
            y1.append(attr_y)
            print(numbercheck, attr_y, "테스트용")
        
    x = np.array(x)
    y = np.array(y)
    x1 = np.array(x1)
    y1 = np.array(y1)

    return x , y, x1, y1

#경로 지정 후 이미지와 라벨 로드.
train_dir = 'data/'
train_images ,train_labels, test_images , test_labels = createdataset(train_dir)

print('data set shape :',train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)

# 라벨 설정
class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']

# 잔처리
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# 배열 저장. 테스트 할 때 배열만 불러서 모델 훈련 가능해요!
path = 'model/'
np.save(path+'saved_train_images', train_images)
np.save(path+'saved_train_labels', train_labels)
np.save(path+'saved_test_images', test_images)
np.save(path+'saved_test_labels', test_labels)

모델 훈련

In [ ]:
import numpy as np

# 배열 불러오기
path = 'model/'
train_images = np.load(path+'saved_train_images.npy')
train_labels = np.load(path+'saved_train_labels.npy')
test_images = np.load(path+'saved_test_images.npy')
test_labels = np.load(path+'saved_test_labels.npy')

In [ ]:
#훈련을 위해 라벨 배열에 원 핫 엔코딩 실시
from tensorflow.keras.utils import to_categorical  # one-hot encoding 을 위한 함수
number_of_classes = 10

# 라벨 2차원으로 보정
train_labels = to_categorical(train_labels, number_of_classes)  # 원-핫 인코딩. 1차원 -> 2차원
test_labels = to_categorical(test_labels, number_of_classes)

print(f"train_labels_shape: {train_labels.shape}") 
print(f"test_labels_shape: {test_labels.shape}") 

In [ ]:
# 데이터 형태 확인
print('data set shape :',train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)

In [ ]:
#모델 훈련
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

# 모델 객체를 만듭니다
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 구조를 출력합니다
model.summary()

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 훈련하기
model.fit(train_images, train_labels,  epochs=30,
          validation_data=(test_images,test_labels),
          batch_size=200,
          verbose=1)  # 콜백을 훈련에 전달합니다

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("모델의 정확도: {:5.2f}%".format(100*acc))

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

# 전체 모델을 저장합니다
model.save('model/my_model.h5')

In [ ]:
# 훈련 결과 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs = 15
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

결과 확인

In [ ]:
"""
검증 1(내부 이미지)
"""

# 0. 사용할 패키지 불러오기
from keras.utils import np_utils
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
import tensorflow as tf

# 1. 실무에 사용할 데이터 준비하기(지금은 테스트 파일에 실험용으로 저장해둔 데이터세트 파일을 불러옵니다.)
test_images = np.load('test/saved_test_images.npy')
test_labels = np.load('test/saved_test_labels.npy')

print('data set shape :',test_images.shape,test_labels.shape)

class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']

test_labels = np_utils.to_categorical(test_labels)
xhat_idx = np.random.choice(test_images.shape[0], 1)
xhat = test_images[xhat_idx]

xhat_image1 = np.resize(xhat, (32, 32))
plt.imshow(xhat_image1)
plt.show

# 2. 모델 불러오기
from keras.models import load_model
model = load_model('test/model.h5')

# 3. 모델 사용하기
yhat = model.predict(xhat)
score = tf.nn.softmax(yhat[0])
print('정답 : ' + class_names[ (argmax(test_labels[xhat_idx[0]])) ] )
print(
    "예측 : {}"
    .format(class_names[np.argmax(score)])
)

In [ ]:
"""
검증 2(외부 이미지)
"""

# 필요 라이브러리 호출
from keras.models import load_model
import cv2 
from google.colab.patches import cv2_imshow
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

# 이미지가 있는 경로를 아래 url위치에 넣어주세요
# 이미지를 불러오고 그레이 스케일로 변환 
src = cv2.imread("test/test9.png",cv2.IMREAD_GRAYSCALE) 
print("이미지 크기 : ",src.shape[1], src.shape[0])

#이진화 1
ret , binary = cv2.threshold(src,127,255,cv2.THRESH_BINARY_INV)

#외곽선 검출
contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)

#하얀색으로 외곽선을 그려준 뒤 이진화하고 리사이즈
cv2.drawContours(binary , contours , -1 , (255,255,255),1)
color = np.ceil(np.array(binary) / 255) * 255
color = cv2.resize(color , (32,32) )

# 변환된 이미지 보기
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

# 데이터를 모델에 적용할 수 있도록 가공
color = np.resize(color, (1, 32, 32, 1))

# 모델 불러오기
model = load_model('model/my_model.h5')

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']
predictions = model.predict(color)

res = np.argmax(predictions[0])
print("결과는 : ", class_names[res])
# 예측 실패 사례.
# test5.png(^, 32X32) 는 +로, test6.png(&, 32X32)는 $로 예측중.
# testimage6.png(&, 40X62)는 $로, testimage8.png(?, 64x53)는 %로 예측중.

##테스트용 코드(연습장)

In [ ]:
#테스트파일이름체크
import os 
import cv2
import numpy as np
from sklearn.preprocessing import * 
import matplotlib.pyplot as plt 

# 데이터 세트에서 훈련용 데이터가 가지는 비율 (90%면 0.9입니다)
rate = 0.9

def createdataset(directory):       #sklearn사용을 위해 데이터세트를 생성

    # 파일 불러오기
    files = os.listdir(directory)

    # 훈련용 배열
    x = []
    y = []

    # 테스트용 베열
    x1 = []
    y1 = []

    # 테스트용 파일의 시작 순번
    file_num = int(len(files) / 10 * rate)
    
    print("저장된 파일 수 : ", len(files))
    print("훈련용 파일의 마지막 순번 : ", file_num)
    print("예상되는 훈련용 파일 수 : ", file_num * 10)

    for file in files:
        print(file)
        attr_x = cv2.imread(directory+file, cv2.IMREAD_GRAYSCALE)
        attr_x = attr_x.reshape(32,32,1)
        attr_y = int(file[0])
        
        #파일 뒤의 숫자를 확인하여 훈련용, 테스트용을 비율로 구분한다.
        numbercheck = int(file.split('_')[1].split('.')[0])
       
        # 훈련용으로 분류
        if numbercheck < file_num : 
            x.append(attr_x)
            y.append(attr_y)
            print(numbercheck, attr_y, "훈련용")

        # 테스트용으로 분류
        else : 
            x1.append(attr_x)
            y1.append(attr_y)
            print(numbercheck, attr_y, "테스트용")
        
    x = np.array(x)
    y = np.array(y)
    
    x1 = np.array(x1)
    y1 = np.array(y1)

    return x , y, x1, y1

#경로 지정 후 이미지와 라벨 로드.
path = 'data/'
train_images ,train_labels, test_images , test_labels = createdataset(path)

print('data set shape :',train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)

# 라벨 설정
class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']

# 잔처리
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))

for i in range(300,400):
    n = i - 300
    plt.subplot(10,10,n+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
#모델 훈련
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# 모델 객체를 만듭니다
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


# 모델 구조를 출력합니다
model.summary()

model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

# 문제는 9000개.
# 모델 훈련하기. 검증 데이터는 0.2 비율로.
history = model.fit(train_images, train_labels, batch_size=300, epochs=10,validation_split=0.2)
loss,acc = model.evaluate(test_images,  test_labels)

print("손실률:", loss)
print("정확도:", acc)


# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

plt.figure(figsize=(18, 6))

# 에포크별 정확도
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.title("accuracy")
plt.legend()

# 에포크별 손실률
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("loss")
plt.legend()

plt.show()

# 전체 모델을 저장합니다
model.save('test/model.h5')

In [ ]:
"""
검증 1(내부 이미지)
"""

# 0. 사용할 패키지 불러오기
from keras.utils import np_utils
import numpy as np
from numpy import argmax

# 1. 실무에 사용할 데이터 준비하기
test_images = np.load('test/saved_test_images.npy')
test_labels = np.load('test/saved_test_labels.npy')

print('data set shape :',test_images.shape,test_labels.shape)

class_names = ['!', '@', '#', '$', '%',
               '^', '&', '*', '?', '+']

test_labels = np_utils.to_categorical(test_labels)
xhat_idx = np.random.choice(test_images.shape[0], 1)
xhat = test_images[xhat_idx]

xhat_image1 = np.resize(xhat, (32, 32))
plt.imshow(xhat_image1)
plt.show

# 2. 모델 불러오기
from keras.models import load_model
model = load_model('test/model.h5')

# 3. 모델 사용하기
yhat = model.predict(xhat)
score = tf.nn.softmax(yhat[0])
print('정답 : ' + class_names[ (argmax(test_labels[xhat_idx[0]])) ] )
print(
    "예측 : {}, 자신 확률 :  {:.2f}% "
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
"""
검증 2(외부 이미지)
"""

# 필요 라이브러리 호출(PIL은 이미지파일 처리위함)
from keras.models import load_model
import cv2 
from google.colab.patches import cv2_imshow
import tensorflow as tf
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

"""
 이미지가 있는 경로를 아래 url위치에 넣어주세요
 이미지를 불러오고 그레이 스케일로 변환 
"""
src = cv2.imread("test/testimage0.png",cv2.IMREAD_GRAYSCALE) 

#이진화 1
ret , binary = cv2.threshold(src,127,255,cv2.THRESH_BINARY_INV)

#외곽선 검출
contours , hierarchy = cv2.findContours(binary , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)

#하얀색으로 외곽선을 그려준 뒤 이진화하고 리사이즈
cv2.drawContours(binary , contours , -1 , (255,255,255),1)
color = np.ceil(np.array(binary) / 255) * 255
color = cv2.resize(color , (32,32) )

# 변환된 이미지 보기
cv2_imshow(color) 
k = cv2.waitKey(0) 
cv2.destroyAllWindows()

# 데이터를 모델에 적용할 수 있도록 가공
color = np.resize(color, (1, 32, 32, 1))

# 모델 불러오기
model = load_model('test/model.h5')
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

# 클래스 예측 함수에 가공된 테스트 데이터 넣어 결과 도출
class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']
predictions = model.predict(color)

score = tf.nn.softmax(predictions[0])
print(
    "예측 : {}, 자신 확률 :  {:.2f}% "
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# 예측 실패 사례(test/model.h5)
# test6.png(&) 는 *로 예측함
# testimage6.png(&)는 $로, testimage8.png(?)는 $로 예측중.

In [ ]:
"""
예측
"""
from sklearn.externals import joblib
from tensorflow.keras.models import load_model
import tensorflow as tf
from PIL import Image
import numpy as np

model = load_model('test/model')
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

##########모델 예측
width = 32
height = 32
color = 1 #색수

file_path = 'test/test2.png'
img = Image.open(file_path).convert('L')
img = image.resize((width, height))

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

##SESS 방식(구버전)

In [ ]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

import random 
import matplotlib.pyplot as plt 

tf.set_random_seed(777) #reproducibility 

"""
모델 구축
"""
X = tf.placeholder(tf.float32, [None, 32,32,1]) 
Y = tf.placeholder(tf.float32, [None, 10]) 
keep_prob = tf.placeholder(tf.float32) 

#=============================================================================== 
W1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev=0.01)) 
#표준편차가 0.01인 3x3매트릭스(깊이1) 데이터를 16개만큼 뽑겠다 
#32x32x16 

L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME') 
#strides를 통해 축을 여러개 만들어 주어 이동방향을 여러방향으로 하게 했으나 가운데 1,1만 적용이 된다 
#제로패딩을통해 사이즈를 동일하게 만듬 
L1 = tf.nn.relu(L1) #relu함수를 이용 음수값을 제거하는 방법으로 사용된다. 
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
#16x16x16 
#컨벌루션된 결과를 max pooling을 통해 최댓값으로 다시 정리 1차원축으로 2칸 2차원축으로도 2칸이동한다 
#제로패딩을 통해 크기가 줄지 않아야하지만 2x2필터를 사용하여서 크기가 반으로 줄어든다 depth는 그대로 유지된다(32) 
#=============================================================================== 
W2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01)) 
#역시 필터를 생성하는데 3x3매트릭스의 깊이가16인 데이터를 32개를 사용한다 
#16x16x32 
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME') 
L2 = tf.nn.relu(L2) 
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
#8x8x32 
#=============================================================================== 
#레이어 추가 부분 
W3 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev =0.01)) 
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME') #7x7x64 
L3 = tf.nn.relu(L3) 
L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
#4x4x64 
#=============================================================================== 
L3_flat = tf.reshape(L3,[-1, 4 * 4 * 64]) 

W4 = tf.Variable(tf.random_normal([4 * 4 * 64, 10],stddev=0.01)) 
# 최종 출력값 L2 에서의 출력 4*4*64개를 입력값으로 받아서 0~9 레이블인 10개의 출력값을 만듬
 #=============================================================================== 
b = tf.Variable(tf.random_normal([10])) 
hypothesis = tf.matmul(L3_flat, W4) +b 
target1 = tf.argmax(tf.nn.softmax(hypothesis),1) 
 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y)) 

 """
모델 학습
 """

#costfunction 
learning_rate = 0.001 #러닝하면서 찾아갈 이동 단위 
training_epochs = 20 #훈련을 시킬 횟수 
batch_size = 64 #훈련시킬 데이터의 묶음 

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 
#adamoptimizer를 사용 adam은 최적값의 근사치일때 넓게 점프해서 한번더 체크해준다 
sess = tf.Session() 
#세션을 통해 메모리에 설정한것들을 올려준다 
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs) : #같은 트레이닝데이터를 epoch수만큼 훈련시킨다 
    avg_cost = 0 #오버피팅을 통한 정확도 상승을 목표로 한다 
    total_batch = int(train_images.shape[0]/ batch_size) 
    #train set에 example들을 batchsize(100)만큼 나누어주어 total batch를 구한다 
    batch_start = 0 
    batch_finish = batch_size 
    
    for i in range(total_batch) : #전체 데이터를 100장씩 훈련시키게 된다 
        batch_xs = train_images[batch_start:batch_finish] 
        batch_ys = train_labels[batch_start:batch_finish] 
        feed_dict = {X:batch_xs, Y:batch_ys} #placeholder로 설정한 그릇에 데이터 담아줌 
        cost_val, _ ,hypo,tg = sess.run([cost,optimizer,hypothesis,target1],feed_dict=feed_dict) 
        avg_cost += cost_val / total_batch #출력을 위해 avg코스트를 만들어주어 확인 
        
        batch_start += total_batch 
        batch_finish += total_batch 
        
        if batch_finish > train_images.shape[0] :
            batch_finish = -1 
            
    print('Epoch :', '%04d'%(epoch+1),'cost = ','{:9f}'.format(avg_cost)) 
    
print('Learning Finished!!')    #5번의 훈련(epoch)을 모두 마치면 출력 

correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) 
print('Accuracy : ', sess.run(accuracy, feed_dict={X:test_images, Y:test_labels})) 

"""
검증
"""

class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']

r = random.randint(0,test_images.shape[0]-1) 
print('Label : ',class_names[ sess.run(tf.argmax(test_labels[r:r+1],1))[0]  ]) 
print('Prediction : ', class_names[ sess.run(tf.argmax(hypothesis,1), feed_dict={X:test_images[r:r+1]})[0] ]) 

plt.imshow(test_images[r:r+1].reshape(32,32),cmap = 'Greys', interpolation='nearest') 
plt.show()

"""
저장
"""

# The file path to save the data
save_file = 'model_sess.ckpt'
 
# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))
 
# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()
 
with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.global_variables_initializer())
 
    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))
 
    # Save the model
    saver.save(sess, save_file)

In [ ]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

import random 
import matplotlib.pyplot as plt 

# 불러오기
sess = tf.Session()
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
saver.restore(sess, 'test/model_sess')

class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']

r = random.randint(0,test_images.shape[0]-1) 
plt.imshow(test_images[r:r+1].reshape(32,32),cmap = 'Greys', interpolation='nearest') 

print('Label : ',class_names[ sess.run(tf.argmax(test_labels[r:r+1],1))[0]  ]) 
print('Prediction : ', class_names[ sess.run(tf.argmax(hypothesis,1), feed_dict={X:test_images[r:r+1]})[0] ]) 
plt.show()

In [ ]:
"""
예측
"""
from sklearn.externals import joblib
from tensorflow.keras.models import load_model
import tensorflow as tf
from PIL import Image
import numpy as np

model = load_model('test/model.h5')

##########모델 예측

file_path = 'test/test2.png'
image = Image.open(file_path).convert('L')

# 변환된 이미지 보기
plt.imshow(image)
plt.show

width = 32
height = 32
color = 1 #색수
image = image.resize((width, height))
image = np.array(image) 
print(image.shape)
image = image.reshape((width, height, color))
print(image.shape)

x_test = [image]
x_test = np.array(x_test)
x_test = x_test / 255

y_predict = model.predict(x_test)
print(y_predict) #[[0.20842288 0.41051054 0.38106653]]
print(y_predict.argmax(axis=1)) #[1]
print(y_predict.argmax(axis=1)[0]) #1
class_names = ['!', '@', '#', '$', '%', '^', '&', '*', '?', '+']
print(class_names[y_predict.argmax(axis=1)[0]])